# are-you-root

okokokokokokokokokokokokokokokokokokokok


## find read code
we read code. see C program

```C
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>

typedef enum auth_level {
  ANONYMOUS = 1,
  GUEST = 2,
  USER = 3,
  ADMIN = 4,
  ROOT = 5
} auth_level_t;

struct user {
  char *name;
  auth_level_t level;
};

void give_flag(){
  char flag[48];
  FILE *f = fopen("flag.txt", "r");
  if (f == NULL) {
    printf("Flag File is Missing. Problem is Misconfigured, please contact an Admin if you are running this on the shell server.\n");
    exit(0);
  }

  if ((fgets(flag, 48, f)) == NULL){
    puts("Couldn't read flag file.");
    exit(1);
  };

  puts(flag);
  fclose(f);
}

void menu(){
  puts("Available commands:");
  puts("\tshow - show your current user and authorization level");
  puts("\tlogin [name] - log in as [name]");
  puts("\tset-auth [level] - set your authorization level (must be below 5)");
  puts("\tget-flag - print the flag (requires authorization level 5)");
  puts("\treset - log out and reset authorization level");
  puts("\tquit - exit the program");
}

int main(int argc, char **argv){
  char buf[512];
  char *arg;
  uint32_t level;
  struct user *user;

  setbuf(stdout, NULL);

  menu();

  user = NULL;
  while(1){
    puts("\nEnter your command:");
    putchar('>'); putchar(' ');

    if(fgets(buf, 512, stdin) == NULL)
      break;

    if (!strncmp(buf, "show", 4)){
      if(user == NULL){
	      puts("Not logged in.");
      }else{
	      printf("Logged in as %s [%u]\n", user->name, user->level);
      }

    }else if (!strncmp(buf, "login", 5)){
      if (user != NULL){
	      puts("Already logged in. Reset first.");
	      continue;
      }

      arg = strtok(&buf[6], "\n");
      if (arg == NULL){
	      puts("Invalid command");
	      continue;
      }

      user = (struct user *)malloc(sizeof(struct user));
      if (user == NULL) {
	      puts("malloc() returned NULL. Out of Memory\n");
	      exit(-1);
      }
      user->name = strdup(arg);
      printf("Logged in as \"%s\"\n", arg);

    }else if(!strncmp(buf, "set-auth", 8)){
      if(user == NULL){
        puts("Login first.");
        continue;
      }

      arg = strtok(&buf[9], "\n");
      if (arg == NULL){
        puts("Invalid command");
        continue;
      }

      level = strtoul(arg, NULL, 10);

      if (level >= 5){
        puts("Can only set authorization level below 5");
        continue;
      }

      user->level = level;
      printf("Set authorization level to \"%u\"\n", level);

    }else if(!strncmp(buf, "get-flag", 8)){
      if (user == NULL){
        puts("Login first!");
        continue;
      }

      if (user->level != 5){
        puts("Must have authorization level 5.");
        continue;
      }

      give_flag();
    }else if(!strncmp(buf, "reset", 5)){
      if (user == NULL){
      puts("Not logged in!");
      continue;
      }

      free(user->name);
      user = NULL;

      puts("Logged out!");
    }else if(!strncmp(buf, "quit", 4)){
      return 0;
    }else{
      puts("Invalid option");
      menu();
    }
  }
}
```

ignore `give_flag` function cuz thats what we want to run. also ignore the `menu` function becasue it isnt taking any user input, and no arguments are passed to it

inital look at the program is "can we convince the program to let us print out the flag normally". the relevant program parts to make this happen are as follows

```C
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>

typedef enum auth_level {
  ANONYMOUS = 1,
  GUEST = 2,
  USER = 3,
  ADMIN = 4,
  ROOT = 5
} auth_level_t;

struct user {
  char *name;
  auth_level_t level;
};

...

int main(int argc, char **argv){
  char buf[512];
  char *arg;
  uint32_t level;
  struct user *user;
...
  while(1){
...

    if(fgets(buf, 512, stdin) == NULL)
      break;
...
    }else if(!strncmp(buf, "set-auth", 8)){
      if(user == NULL){
        puts("Login first.");
        continue;
      }

      arg = strtok(&buf[9], "\n");
      if (arg == NULL){
        puts("Invalid command");
        continue;
      }

      level = strtoul(arg, NULL, 10);

      if (level >= 5){
        puts("Can only set authorization level below 5");
        continue;
      }

      user->level = level;
      printf("Set authorization level to \"%u\"\n", level);

    }else if(!strncmp(buf, "get-flag", 8)){
      if (user == NULL){
        puts("Login first!");
        continue;
      }

      if (user->level != 5){
        puts("Must have authorization level 5.");
        continue;
      }

      give_flag();
    }
...
  }
}
```


specifically, the parts we are interested in about this is the items that allow us to get to the execution of the `give_flag` function. in order to get there, we need to have a `user->level` of 5. however, we see that the only place where `level` can be set is in the "set-auth" case, and you can only set the value of level to be some value less than 5. so via normal means, this will not work

## what can we break???

so theres a few things we might explore as to what to do next. what we are looking for is things that we can do that will lead to us getting the `give_flag` function to run, and i guess the next step is just find things that might "break" or be interpreted wrong, and hopefully we can use that in a path leading to a `give_flag` execution

i noticed that the `level` variable in `main` is `uint32_t`, but the `auth_level` in the `user` struct is an enum. according to this [stack overflow question](https://stackoverflow.com/questions/159034/are-c-enums-signed-or-unsigned#:~:text=In%20short%3A%20you%20cannot%20rely,being%20either%20signed%20or%20unsigned.&text=Michael%20Burr's%20answer%20(which%20quotes,values%20defined%20in%20the%20enumeration%22.), theres a potential that the enum is stored as a `int` instead of a `uint`. unfortunately

1) we are using small positive values, so the bit representation of the numbers we are working with would have the same unsinged and signed binary encodings
2) user input is being read and converted to an unsigned long, and then stored into a unsigned int if the value < 5. 
3) we never actually reference the enum values, and just use it as a data type to the `user` struct

---

seeing as that wont work, we look for other uses

what we are going to try doing is evaluating each conditional branch/path of the program as like a building block, and 
